<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_3/13%20-%20Atividade%20Avalaitiva/atividade_avaliativa_modulo_6_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Segmentação de Imagens de Dígitos Manuscritos

<img src='https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/adv_cv/class_3/13%20-%20Atividade%20Avalaitiva/images/m2nist_segmentation.png' alt='m2nist digits'>

Nessa atividade você criará um modelo que prevê as máscaras de segmentação (mapa de rótulos por pixel) de dígitos manuscritos. Esse modelo será treinado no [M2NIST dataset](https://www.kaggle.com/farhanhubble/multimnistm2nist), um MNIST de vários dígitos.

Note que muitas das etapas aqui são muito semelhantes ao que você viu na aula [6.8-Redes neurais totalmente convolucionais para segmentação de imagens](https://drive.google.com/file/d/1XLkmlyzmP95mDtArmZXhvA_oJBGgKWZh/view?usp=drive_link).

Você criará uma rede neural convolucional (CNN) do zero para o caminho de downsampling e usará uma rede totalmente convolucional, a FCN-8, para fazer o upsample e produzir o mapa de rótulos em pixels. O modelo será avaliado usando a interseção sobre a união (IOU) e o Dice Score.

## Exercícios

Forneço a você um código padrão para trabalhar e estes são os 5 exercícios que você precisa preencher antes de obter as máscaras de segmentação com êxito.

* [Exercício 1 - Definir o Bloco Convolucional Básico](#exercise-1)
* [Exercício 2 - Definir o Caminho de Downsampling](#exercise-2)
* [Exercício 3 - Definir o decoder FCN-8](#exercise-3)
* [Exercício 4 - Compilação do Modelo](#exercise-4)
* [Exercício 5 - Treino do Modelo](#exercise-5)

## Importações

Como de costume, vamos começar importando os pacotes que você usará nesta atividade.

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import zipfile

import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

print("Tensorflow version " + tf.__version__)

## Download do dataset

[M2NIST](https://www.kaggle.com/farhanhubble/multimnistm2nist) é um [MNIST](http://yann.lecun.com/exdb/mnist/) **multidígito**.
Cada imagem tem até 3 dígitos de dígitos do MNIST e o arquivo de rótulos correspondente tem as máscaras de segmentação.

O conjunto de dados está disponível no [Kaggle](https://www.kaggle.com) e você pode encontrá-lo [aqui](https://www.kaggle.com/farhanhubble/multimnistm2nist)

Para facilitar, ele está hospedado no Google Cloud para que você possa fazer o download sem as credenciais do Kaggle.

In [ ]:
# download do dataset zipado
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/m2nist.zip \
    -O /tmp/m2nist.zip

# localizar e extrair para uma pasta local ('/tmp/training')
local_zip = '/tmp/m2nist.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/training')
zip_ref.close()

## Carregar e Preprocessar o Dataset

Esse conjunto de dados pode ser facilmente pré-processado, pois está disponível como **Numpy Array Files (.npy)**

1. O **combined.npy** tem os arquivos de imagem que contêm os vários dígitos do MNIST. Cada imagem tem o tamanho **64 x 84** (altura x largura, em pixels).

2. O arquivo **segmented.npy** contém as máscaras de segmentação correspondentes. Cada máscara de segmentação também tem o tamanho de **64 x 84**.

Esse conjunto de dados tem **5000** amostras e você pode fazer divisões apropriadas de treinamento, validação e teste, conforme necessário para o problema.

Com isso, vamos definir algumas funções utilitárias para carregar e pré-processar o conjunto de dados.

In [ ]:
BATCH_SIZE = 32

def read_image_and_annotation(image, annotation):
  '''
  Converte a imagem e a anotação em seu tipo de dados esperado e
  normaliza a imagem de entrada de modo que cada pixel esteja no intervalo [-1, 1]

  Args:
    image (matriz numpy) -- imagem de entrada
    annotation (matriz numpy) -- mapa de rótulos de ground-truth

  Retorna:
    par imagem-anotação pré-processado
  '''

  image = tf.cast(image, dtype=tf.float32)
  image = tf.reshape(image, (image.shape[0], image.shape[1], 1,))
  annotation = tf.cast(annotation, dtype=tf.int32)
  image = image / 127.5
  image -= 1

  return image, annotation


def get_training_dataset(images, annos):
  '''
  Prepara lotes embaralhados do conjunto de treinamento.

  Args:
    images (lista de strings) -- caminhos para cada arquivo de imagem no conjunto de treinamento
    annos (lista de strings) -- caminhos para cada mapa de rótulo no conjunto de treinamento

  Retorna:
    tf Conjunto de dados contendo o conjunto de treinamento pré-processado
  '''
  training_dataset = tf.data.Dataset.from_tensor_slices((images, annos))
  training_dataset = training_dataset.map(read_image_and_annotation)

  training_dataset = training_dataset.shuffle(512, reshuffle_each_iteration=True)
  training_dataset = training_dataset.batch(BATCH_SIZE)
  training_dataset = training_dataset.repeat()
  training_dataset = training_dataset.prefetch(-1)

  return training_dataset


def get_validation_dataset(images, annos):
  '''
  Prepara os lotes do conjunto de validação.

  Args:
    images (lista de strings) -- caminhos para cada arquivo de imagem no val set
    annos (lista de strings) -- caminhos para cada mapa de rótulo no conjunto de valores

  Retorna:
    tf Conjunto de dados contendo o conjunto de validação pré-processado
  '''
  validation_dataset = tf.data.Dataset.from_tensor_slices((images, annos))
  validation_dataset = validation_dataset.map(read_image_and_annotation)
  validation_dataset = validation_dataset.batch(BATCH_SIZE)
  validation_dataset = validation_dataset.repeat()

  return validation_dataset


def get_test_dataset(images, annos):
  '''
  Prepara os lotes do conjunto de teste.

  Args:
    images (lista de strings) -- caminhos para cada arquivo de imagem no conjunto de teste
    annos (lista de strings) -- caminhos para cada mapa de rótulo no conjunto de teste

  Retorna:
    tf Conjunto de dados contendo o conjunto de validação pré-processado
  '''
  test_dataset = tf.data.Dataset.from_tensor_slices((images, annos))
  test_dataset = test_dataset.map(read_image_and_annotation)
  test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

  return test_dataset


def load_images_and_segments():
  '''
  Carrega as imagens e os segmentos como matrizes numpy de arquivos npy
  e faz divisões para conjuntos de dados de treinamento, validação e teste.

  Retorna:
    3 tuplas contendo as divisões de treinamento, validação e teste
  '''

  #Carrega imagens e máscaras de segmentação.
  images = np.load('/tmp/training/combined.npy')
  segments = np.load('/tmp/training/segmented.npy')

  #Faz divisões de treinamento, validação e teste a partir de imagens carregadas e máscaras de segmentação.
  train_images, val_images, train_annos, val_annos = train_test_split(images, segments, test_size=0.2, shuffle=True)
  val_images, test_images, val_annos, test_annos = train_test_split(val_images, val_annos, test_size=0.2, shuffle=True)

  return (train_images, train_annos), (val_images, val_annos), (test_images, test_annos)


Agora você pode carregar o conjunto de dados pré-processado e definir os conjuntos de treino, validação e teste.

In [ ]:
# Carregar o Dataset
train_slices, val_slices, test_slices = load_images_and_segments()

# Criar os datasets de treino, validação e teste.
training_dataset = get_training_dataset(train_slices[0], train_slices[1])
validation_dataset = get_validation_dataset(val_slices[0], val_slices[1])
test_dataset = get_test_dataset(test_slices[0], test_slices[1])

## Vamos dar uma olhada no dataset

Talvez você queira inspecionar visualmente o dataset antes e depois do treino. Como acima, incluí funções utilitárias para ajudar a mostrar algumas imagens, bem como suas anotações (ou seja, rótulos).

In [ ]:
# Utilitários de visualização

# Existem 11 classes no conjunto de dados: uma classe para cada dígito (0 a 9)
# mais a classe de fundo
n_classes = 11

# Atribuir uma cor aleatória para cada classe
colors = [tuple(np.random.randint(256, size=3) / 255.0) for i in range(n_classes)]

def fuse_with_pil(images):
  '''
  Cria uma imagem em branco e cola as imagens de entrada

  Args:
    images (lista de matrizes numpy) - representações de matrizes numpy das imagens a serem coladas

  Retorna:
    Objeto PIL Image contendo as imagens
  '''
  widths = (image.shape[1] for image in images)
  heights = (image.shape[0] for image in images)
  total_width = sum(widths)
  max_height = max(heights)

  new_im = PIL.Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    pil_image = PIL.Image.fromarray(np.uint8(im))
    new_im.paste(pil_image, (x_offset,0))
    x_offset += im.shape[1]

  return new_im


def give_color_to_annotation(annotation):
  '''
  Converte uma anotação 2-D em uma matriz numpy com formato (altura, largura, 3) em que
  o terceiro eixo representa o canal de cor. Os valores do rótulo são multiplicados por
  255 e colocados nesse eixo para dar cor à anotação

  Args:
    annotation (matriz numpy) - matriz do mapa de rótulos

  Retorna:
    a matriz de anotação com um canal/eixo de cor adicional
  '''
  seg_img = np.zeros( (annotation.shape[0],annotation.shape[1], 3) ).astype('float')

  for c in range(n_classes):
    segc = (annotation == c)
    seg_img[:,:,0] += segc*( colors[c][0] * 255.0)
    seg_img[:,:,1] += segc*( colors[c][1] * 255.0)
    seg_img[:,:,2] += segc*( colors[c][2] * 255.0)

  return seg_img


def show_annotation_and_prediction(image, annotation, prediction, iou_list, dice_score_list):
  '''
  Exibe as imagens com o ground-truth e os mapas de rótulos previstos. Também sobrepõe as métricas.

  Args:
    image (matriz numpy) -- a imagem de entrada
    annotation (matriz numpy) -- o mapa de rótulos da verdade terrestre
    prediction (matriz numpy) -- o mapa de rótulos previsto
    iou_list (lista de floats) -- os valores de IOU para cada classe
    dice_score_list (lista de floats) -- a pontuação de dados para cada classe
  '''
  new_ann = np.argmax(annotation, axis=2)
  true_img = give_color_to_annotation(new_ann)
  pred_img = give_color_to_annotation(prediction)

  image = image + 1
  image = image * 127.5
  image = np.reshape(image, (image.shape[0], image.shape[1],))
  image = np.uint8(image)
  images = [image, np.uint8(pred_img), np.uint8(true_img)]

  metrics_by_id = [(idx, iou, dice_score) for idx, (iou, dice_score) in enumerate(zip(iou_list, dice_score_list)) if iou > 0.0 and idx < 10]
  metrics_by_id.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place

  display_string_list = ["{}: IOU: {} Dice Score: {}".format(idx, iou, dice_score) for idx, iou, dice_score in metrics_by_id]
  display_string = "\n".join(display_string_list)

  plt.figure(figsize=(15, 4))

  for idx, im in enumerate(images):
    plt.subplot(1, 3, idx+1)
    if idx == 1:
      plt.xlabel(display_string)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(im)


def show_annotation_and_image(image, annotation):
  '''
  Exibe a imagem e sua anotação lado a lado

  Args:
    image (matriz numpy) -- a imagem de entrada
    annotation (numpy array) -- o mapa de rótulos
  '''
  new_ann = np.argmax(annotation, axis=2)
  seg_img = give_color_to_annotation(new_ann)

  image = image + 1
  image = image * 127.5
  image = np.reshape(image, (image.shape[0], image.shape[1],))

  image = np.uint8(image)
  images = [image, seg_img]

  images = [image, seg_img]
  fused_img = fuse_with_pil(images)
  plt.imshow(fused_img)


def list_show_annotation(dataset, num_images):
  '''
  Exibe imagens e suas anotações lado a lado

  Args:
    dataset (tf Dataset) -- lote de imagens e anotações
    num_images (int) -- número de imagens a serem exibidas
  '''
  ds = dataset.unbatch()

  plt.figure(figsize=(20, 15))
  plt.title("Imagens e Anotações")
  plt.subplots_adjust(bottom=0.1, top=0.9, hspace=0.05)

  for idx, (image, annotation) in enumerate(ds.take(num_images)):
    plt.subplot(5, 5, idx + 1)
    plt.yticks([])
    plt.xticks([])
    show_annotation_and_image(image.numpy(), annotation.numpy())


Você pode visualizar um subconjunto de imagens do conjunto de dados com a função `list_show_annotation()` definida acima. Execute as células abaixo para ver a imagem à esquerda e seu mapa de rótulos de ground-truth em pixels à direita.

In [ ]:
# Obter 10 imagens do conjunto de treinamento
list_show_annotation(training_dataset, 10)

In [ ]:
# Obter 10 imagens do conjunto de validação
list_show_annotation(validation_dataset, 10)

Você pode ver nas imagens acima as cores atribuídas a cada classe (ou seja, de 0 a 9 mais o plano de fundo). Se você não gostar dessas cores, sinta-se à vontade para executar novamente a célula em que `colors` está definida para obter outro conjunto de cores aleatórias. Como alternativa, você pode atribuir os valores RGB para cada classe em vez de depender de valores aleatórios.

## Definir o modelo

O modelo de segmentação tem doi caminhos:

1. **Caminho de Downsampling** - Essa parte da rede extrai as características da imagem. Isso é feito por meio de uma série de camadas de convolução e agrupamento. A saída final é uma imagem reduzida (devido às camadas de pooling) com as características extraídas. Você criará uma CNN personalizada do zero para esse caminho.

2. **Caminho de Upsampling** - Esse caminho pega a saída do caminho de downsampling e gera as previsões, além de converter a imagem de volta ao seu tamanho original. Você usará um decodificador FCN-8 para esse caminho.

### Definir o bloco de convolução básico

<a name='exercise-1'></a>

#### **Exercício 1**

Preencha a função abaixo para criar o bloco de convolução básico para nossa CNN. Isso terá duas camadas [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/), cada uma seguida por uma [LeakyReLU](https://keras.io/api/layers/activation_layers/leaky_relu/), depois [max pooled](https://keras.io/api/layers/pooling_layers/max_pooling2d/) e [batch-normalized](https://keras.io/api/layers/normalization_layers/batch_normalization/). Use a sintaxe funcional para empilhar essas camadas.

$$Input -> Conv2D -> LeakyReLU -> Conv2D -> LeakyReLU -> MaxPooling2D -> BatchNormalization$$

Ao definir as camadas Conv2D, observe que nossas entradas de dados terão a dimensão "channels" por último. Talvez você queira verificar o argumento `data_format` no [docs](https://keras.io/api/layers/convolution_layers/convolution2d/) com relação a isso. Observe também o argumento `padding` como você fez nos laboratórios não avaliados.

Por fim, para usar a ativação `LeakyReLU`, você **não** precisa aninhá-la dentro de uma camada `Activation` (por exemplo, `x = tf.keras.layers.Activation(tf.keras.layers.LeakyReLU()(x)`). Em vez disso, você pode simplesmente empilhar a camada diretamente (por exemplo, `x = tf.keras.layers.LeakyReLU()(x)`)

In [ ]:
# Parâmetro que descreve onde a dimensão do canal é encontrada em nosso conjunto de dados
IMAGE_ORDERING = 'channels_last'

def conv_block(input, filters, kernel_size, pooling_size, pool_strides):
  '''
  Args:
    input (tensor) -- lote de imagens ou recursos
    filters (int) -- número de filtros das camadas Conv2D
    kernel_size (int) -- configuração de kernel_size das camadas Conv2D
    pooling_size (int) -- tamanho do pooling das camadas MaxPooling2D
    pool_strides (int) -- configuração de strides das camadas MaxPooling2D

  Retorna:
    (tensor) máximo de recursos agrupados e normalizados por lote da entrada
  '''
  ### COMECE O CÓDIGO AQUI ###
  # use a sintaxe funcional para empilhar as camadas, conforme mostrado no diagrama acima  x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', data_format=IMAGE_ORDERING)(input)
  x = None
  x = None
  x = None
  x = None
  x = None
  ### TERMINE O CÓDIGO AQUI ###

  return x

In [ ]:
# CÓDIGO DE TESTE:

test_input = tf.keras.layers.Input(shape=(64,84, 1))
test_output = conv_block(test_input, 32, 3, 2, 2)
test_model = tf.keras.Model(inputs=test_input, outputs=test_output)

print(test_model.summary())

# Liberar recursos de teste
del test_input, test_output, test_model

**Expected Output**:

Preste atenção nas colunas *(type)* e *Output Shape*. O nome *Layer* ao lado do tipo pode ser diferente dependendo de quantas vezes você executou a célula (por exemplo, `input_7` pode ser `input_1`)

```txt
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
=================================================================
input_1 (InputLayer)         [(None, 64, 84, 1)]       0
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 84, 32)        320
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 84, 32)        0
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 84, 32)        9248
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 84, 32)        0
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 42, 32)        0
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 42, 32)        128
=================================================================
Total params: 9,696
Trainable params: 9,632
Non-trainable params: 64
_________________________________________________________________
None
```

### Definir o Caminho de Downsampling

<a name='exercise-2'></a>

#### **Exercício 2**

Agora que definimos o bloco de construção do nosso codificador, você pode criar o caminho de downsampling. Preencha a função abaixo para criar o codificador. Isso deve encadear cinco blocos de construção de convolução para criar uma CNN de extração de características sem as camadas totalmente conectadas.

*Observações*:
1. Para otimizar o processamento ou para facilitar o trabalho com as dimensões de saída de cada camada, às vezes é aconselhável aplicar um pouco de preenchimento zero à imagem de entrada. Com o código padrão que fornecemos abaixo, preenchemos a largura da entrada com 96 pixels usando a camada [ZeroPadding2D](https://keras.io/api/layers/reshaping_layers/zero_padding2d/).No entanto, isso não é obrigatório. Você pode removê-la mais tarde e ver como isso afetará seus parâmetros. Por exemplo, talvez você precise passar um tamanho de kernel não quadrado para o decodificador no Exercício 3 (por exemplo, `(4,5)`) para corresponder às dimensões de saída do Exercício 2.

2. Recomendo manter os parâmetros pool size e stride constantes em 2.

In [ ]:
def FCN8(input_height=64, input_width=84):
    '''
    Define o caminho de downsampling do modelo de segmentação de imagem.

    Args:
      input_height (int) -- altura das imagens
      width (int) -- largura das imagens

    Retorna:
    (tupla de tensores, tensor)
      tupla de tensores -- recursos extraídos nos blocos 3 a 5
      tensor -- cópia da entrada
    '''
    img_input = tf.keras.layers.Input(shape=(input_height,input_width, 1))

    ### COMECE O CÓDIGO AQUI ###

    # Preencher a largura da imagem de entrada com 96 pixels
    x = tf.keras.layers.ZeroPadding2D(((0, 0), (0, 96-input_width)))(img_input)

    # Bloco 1
    x = None

    # Bloco 2
    x = None

    # Bloco 3
    x = None
    # salvar o mapa de características nesse estágio
    f3 = None

    # Bloco 4
    x = None
    # salvar o mapa de características nesse estágio
    f4 = None

    # Bloco 5
    x = None
    # salvar o mapa de características nesse estágio
    f5 = None

    ### TERMINE O CÓDIGO AQUI ###

    return (f3, f4, f5), img_input

In [ ]:
# CÓDIGO DE TESTE:

test_convs, test_img_input = FCN8()
test_model = tf.keras.Model(inputs=test_img_input, outputs=[test_convs, test_img_input])

print(test_model.summary())

del test_convs, test_img_input, test_model

**Resultado esperado**:

Você deverá ver as camadas do seu `conv_block()` sendo repetidas 5 vezes, como na saída abaixo.

```txt
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
=================================================================
input_3 (InputLayer)         [(None, 64, 84, 1)]       0
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 64, 96, 1)         0
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 96, 32)        320
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 96, 32)        0
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 96, 32)        9248
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 64, 96, 32)        0
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 48, 32)        0
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 48, 32)        128
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 48, 64)        18496
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32, 48, 64)        0
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 48, 64)        36928
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 48, 64)        0
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 24, 64)        0
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 24, 64)        256
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 24, 128)       73856
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 24, 128)       0
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 24, 128)       147584
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 16, 24, 128)       0
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 12, 128)        0
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 12, 128)        512
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 12, 256)        295168
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 8, 12, 256)        0
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 12, 256)        590080
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 8, 12, 256)        0
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 6, 256)         0
_________________________________________________________________
batch_normalization_4 (Batch (None, 4, 6, 256)         1024
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 6, 256)         590080
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 4, 6, 256)         0
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 6, 256)         590080
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 4, 6, 256)         0
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 3, 256)         0
_________________________________________________________________
batch_normalization_5 (Batch (None, 2, 3, 256)         1024
=================================================================
Total params: 2,354,784
Trainable params: 2,353,312
Non-trainable params: 1,472
_________________________________________________________________
None
```

### Definir o decodificador FCN-8

<a name='exercise-3'></a>

#### **Exercício 3**

Agora você pode definir o caminho de upsampling usando as saídas das convoluções em cada estágio como argumentos.
* Observação: lembre-se de definir o parâmetro `data_format` para as camadas Conv2D.

Aqui está também o diagrama que você viu na aula sobre como isso deve funcionar:

<img src='https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/adv_cv/class_3/13%20-%20Atividade%20Avalaitiva/images/fcn8.png' alt='fcn-8'>

In [ ]:
def fcn8_decoder(convs, n_classes):
  # características do estágio do codificador
  f3, f4, f5 = convs

  # Número de filtros
  n = 512

  # Adicionar camadas convolucionais sobre o extrator CNN.
  o = tf.keras.layers.Conv2D(n , (7 , 7) , activation='relu' , padding='same', name="conv6", data_format=IMAGE_ORDERING)(f5)
  o = tf.keras.layers.Dropout(0.5)(o)

  o = tf.keras.layers.Conv2D(n , (1 , 1) , activation='relu' , padding='same', name="conv7", data_format=IMAGE_ORDERING)(o)
  o = tf.keras.layers.Dropout(0.5)(o)

  o = tf.keras.layers.Conv2D(n_classes,  (1, 1), activation='relu' , padding='same', data_format=IMAGE_ORDERING)(o)


  ### INICIE O CÓDIGO AQUI

  # Faça o upsample do `o` acima e corte os pixels extras introduzidos
  o = None
  o = None

  # carregue a previsão do pool 4 e faça uma convolução 1x1 para remodelá-la para a mesma forma do `o` acima
  o2 = None
  o2 = None

  # adicione os resultados do upsampling e da previsão do pool 4
  o = None

  # Upsampling do tensor resultante da operação que você acabou de fazer
  o = None
  o = None

  # Carregue a previsão do pool 3 e faça uma convolução 1x1 para remodelá-la para a mesma forma do `o` acima
  o2 = None
  o2 = tf.keras.layers.Conv2D(n_classes , ( 1 , 1 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING)(o2)

  # adicione os resultados da do upsampling e da previsão do pool 3
  o = None

  # Upsampling até o tamanho da imagem original
  o = None
  o = tf.keras.layers.Cropping2D(((0, 0), (0, 96-84)))(o)

  # Acrescentar uma ativação sigmoide
  o = (tf.keras.layers.Activation('sigmoid'))(o)
  ### TERMINE O CÓDIGO AQUI ###

  return o

In [ ]:
# CÓDIGO DE TESTE

test_convs, test_img_input = FCN8()
test_fcn8_decoder = fcn8_decoder(test_convs, 11)

print(test_fcn8_decoder.shape)

del test_convs, test_img_input, test_fcn8_decoder

**Saída esperada:**

```txt
(None, 64, 84, 11)
```

### Definir o modelo completo

Os caminhos de downsampling e upsampling agora podem ser combinados conforme mostrado abaixo.

In [ ]:
# iniciar o codificador usando o tamanho de entrada padrão 64 x 84
convs, img_input = FCN8()

# Passar as convoluções obtidas no codificador para o decodificador
dec_op = fcn8_decoder(convs, n_classes)

# Definir o modelo especificando a entrada (lote de imagens) e a saída (saída do decodificador)
model = tf.keras.Model(inputs = img_input, outputs = dec_op)

In [ ]:
model.summary()

## Compile o Modelo

<a name='exercise-4'></a>

### **Exercício 4**

Compile o modelo usando uma perda, um otimizador e uma métrica apropriados.

Nota:** Atualmente, há um problema com o classificador que aceita determinadas funções de perda. Estaremos atualizando-o, mas, enquanto isso, use esta sintaxe:_

```
loss='<loss string name>'
```

*ao invés de:*

```
loss=tf.keras.losses.<StringCassName>
```



In [ ]:
### START CODE HERE ###
model.compile(loss=None, optimizer=None, metrics=None)
### END CODE HERE ###

## Treino do Modelo

<a name='exercise-5'></a>

### **Exercício 5**

Agora você pode treinar o modelo. Defina o número de épocas e observe as métricas retornadas em cada iteração. Você também pode encerrar a execução da célula se achar que o modelo já está funcionando bem.

In [ ]:
# ALÉM DE DEFINIR O NÚMERO DE ÉPOCAS, NÃO ALTERE NENHUM OUTRO CÓDIGO

### INICIE O CÓDIGO AQUI ###
EPOCHS = None
### TERMINE O CÓDIGO AQUI ###

steps_per_epoch = 4000//BATCH_SIZE
validation_steps = 800//BATCH_SIZE
test_steps = 200//BATCH_SIZE


history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps, epochs=EPOCHS)

**Resultado esperado:**

Em geral, as perdas devem estar diminuindo e a precisão deve estar aumentando. Por exemplo, observar as primeiras 4 épocas deve gerar algo semelhante:

```txt
Epoch 1/70
125/125 [==============================] - 6s 50ms/step - loss: 0.5542 - accuracy: 0.8635 - val_loss: 0.5335 - val_accuracy: 0.9427
Epoch 2/70
125/125 [==============================] - 6s 47ms/step - loss: 0.2315 - accuracy: 0.9425 - val_loss: 0.3362 - val_accuracy: 0.9427
Epoch 3/70
125/125 [==============================] - 6s 47ms/step - loss: 0.2118 - accuracy: 0.9426 - val_loss: 0.2592 - val_accuracy: 0.9427
Epoch 4/70
125/125 [==============================] - 6s 47ms/step - loss: 0.1782 - accuracy: 0.9431 - val_loss: 0.1770 - val_accuracy: 0.9432
```

## Avaliação do Modelo

### Fazer previsões

Vamos obter as previsões usando nosso conjunto de dados de teste como entrada e imprimir a forma.

In [ ]:
results = model.predict(test_dataset, steps=test_steps)

print(results.shape)

Como você pode ver, a forma resultante é `(192, 64, 84, 11)`. Isso significa que, para cada uma das 192 imagens que temos em nosso conjunto de teste, há 11 previsões geradas (ou seja, uma para cada classe: 0 a 1 mais o plano de fundo).

Portanto, se você quiser ver a *probabilidade* de o pixel superior esquerdo da 1ª imagem pertencer à classe 0, poderá imprimir algo como `results[0,0,0,0]`. Se quiser ver a probabilidade de o mesmo pixel pertencer à classe 10, então imprima `results[0,0,0,10]`.

In [ ]:
print(results[0,0,0,0])
print(results[0,0,0,10])

What we're interested in is to get the *index* of the highest probability of each of these 11 slices and combine them in a single image. We can do that by getting the [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) at this axis.

In [ ]:
results = np.argmax(results, axis=3)

print(results.shape)

A nova matriz gerada por imagem agora especifica apenas os índices da classe com a maior probabilidade. Vejamos a classe de saída do pixel superior esquerdo. Como você deve ter observado anteriormente quando inspecionou o conjunto de dados, o canto superior esquerdo geralmente é apenas parte do plano de fundo (classe 10). Os dígitos reais estão escritos em algum lugar nas partes centrais da imagem.

In [ ]:
print(results[0,0,0])

# Mapa de previsão para a imagem 0
print(results[0,:,:])

Usaremos esse array `results` quando avaliarmos nossas previsões.

### Métricas

Nas aulas, mostramos duas maneiras de avaliar suas previsões. A *intersecção sobre união (IOU)* e a *dice score*. Lembre-se de que:


$$IOU = \frac{area\_of\_overlap}{area\_of\_union}$$
<br>
$$Dice Score = 2 * \frac{area\_of\_overlap}{combined\_area}$$

O código abaixo faz isso para você. Um pequeno fator de suavização é introduzido nos denominadores para evitar uma possível divisão por zero.

In [ ]:
def class_wise_metrics(y_true, y_pred):
  '''
  Calcula o IOU e a pontuação de dados por classe.

  Args:
    y_true (tensor) - mapas de rótulos de grouns-truth
    y_pred (tensor) - mapas de rótulos previstos
  '''
  class_wise_iou = []
  class_wise_dice_score = []

  smoothing_factor = 0.00001

  for i in range(n_classes):
    intersection = np.sum((y_pred == i) * (y_true == i))
    y_true_area = np.sum((y_true == i))
    y_pred_area = np.sum((y_pred == i))
    combined_area = y_true_area + y_pred_area

    iou = (intersection) / (combined_area - intersection + smoothing_factor)
    class_wise_iou.append(iou)

    dice_score =  2 * ((intersection) / (combined_area + smoothing_factor))
    class_wise_dice_score.append(dice_score)

  return class_wise_iou, class_wise_dice_score


### Visualize Predictions (Visualizar previsões)

In [ ]:
# Coloque um número aqui entre 0 e 191 para escolher uma imagem do conjunto de teste
integer_slider = 105

ds = test_dataset.unbatch()
ds = ds.batch(200)
images = []

y_true_segments = []
for image, annotation in ds.take(2):
  y_true_segments = annotation
  images = image


iou, dice_score = class_wise_metrics(np.argmax(y_true_segments[integer_slider], axis=2), results[integer_slider])
show_annotation_and_prediction(image[integer_slider], annotation[integer_slider], results[integer_slider], iou, dice_score)


### Calcule a pontuação IOU e Dice Score do seu modelo

In [ ]:
cls_wise_iou, cls_wise_dice_score = class_wise_metrics(np.argmax(y_true_segments, axis=3), results)

average_iou = 0.0
for idx, (iou, dice_score) in enumerate(zip(cls_wise_iou[:-1], cls_wise_dice_score[:-1])):
  print("Dígito {}: IOU: {} Dice Score: {}".format(idx, iou, dice_score))
  average_iou += iou

grade = average_iou * 10

print("\nPontuação é " + str(grade))

PASSING_GRADE = 60
if (grade>PASSING_GRADE):
  print("Passou!")
else:
  print("Não Passou.Verifique seu modelo e treine novamente")

**Parabéns por ter concluído este trabalho sobre segmentação de imagens!